In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.11.8


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
# Q1 What's the standard deviation of the predicted duration for this dataset?

print(y_pred.std())

6.247488852238703


In [12]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
# df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
# df_result['PULocationID'] = df['PULocationID']
# df_result['DOLocationID'] = df['DOLocationID']
# df_result['actual_duration'] = df['duration']
df_result['predicted_duration'] = y_pred
# df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
# df_result['model_version'] = run_id

output_file = f'yellow_{year:04d}-{month:02d}.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [14]:
# Q2 What's the size of the output file?
# 66M

from pathlib import Path
Path(output_file).stat().st_size/1024/1024

65.46199798583984

In [ ]:
# Q3 Now let's turn the notebook into a script. 

# Which command you need to execute for that?

`python predict.py`